In [73]:
import pgeocode

import warnings

import pandas as pd
import numpy as np
import time


countries = pgeocode.COUNTRIES_VALID
countries.insert(0, countries.pop(countries.index("US")))

df = pd.read_csv("../cust.csv")

df = df.dropna(subset=["Ship_Zip"])

In [74]:
zipcodes = df["Ship_Zip"]

Keep only the useful part of the zipcode

In [75]:
zipcodes = np.where(
    df["Ship_Zip"].str.len() == 5, df["Ship_Zip"], df["Ship_Zip"].str[:5]
)

zipcodes = sorted(zipcodes)

In [79]:
zipcode_lookup = {}

for zipcode in zipcodes:
    if zipcode not in zipcode_lookup:
        for i in countries:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                nomi = pgeocode.Nominatim(i)
                result = nomi.query_postal_code(zipcode)
                if not np.isnan(result["accuracy"]):
                    break

        if not np.isnan(result["accuracy"]):
            zipcode_lookup[zipcode] = result.replace(np.nan, None).to_dict()

        if len(zipcode_lookup) % 1000 == 0:
            print(f"Sleeping for 3 seconds, {len(zipcode_lookup)} zipcodes processed")
            time.sleep(3)

Sleeping for 3 seconds, 1000 zipcodes processed
Sleeping for 3 seconds, 2000 zipcodes processed
Sleeping for 3 seconds, 3000 zipcodes processed
Sleeping for 3 seconds, 4000 zipcodes processed
Sleeping for 3 seconds, 5000 zipcodes processed
Sleeping for 3 seconds, 6000 zipcodes processed
Sleeping for 3 seconds, 7000 zipcodes processed
Sleeping for 3 seconds, 8000 zipcodes processed
Sleeping for 3 seconds, 9000 zipcodes processed
Sleeping for 3 seconds, 10000 zipcodes processed


In [90]:
import json

with open("../zipcode_lookup.json", "w") as f:
    json.dump(zipcode_lookup, f)